# Load package

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [ ]:
## 시각화 한글 인식
from matplotlib import font_manager, rc
font_fname = 'C:\\Windows\\Fonts\\malgun.ttf'
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

# Data Load

In [ ]:
!dir

In [ ]:
## 데이터 읽기
data = pd.read_csv('pokemon.csv')
print(data.shape)
data.head(3)

In [ ]:
## column 기본 정보살피기
data.info()

### Results
* 주어진 데이터의**정의**에서는 모든 변수들은 NaN값이 없다.
* 하지만 몇가지 변수들이 NaN값을 포함한게 보인다. 이에 대한 탐구가 필요하다.
* Metadata탐색결과 "height_m","type2","weight_kg"의 null 값은 **단순 결측치**이다. 
* "percentage_male"의 NaN 값은 **중성**을 의미한다.
* 모든 값을 치환한 후 **"이상값(Outlier)"**의 확인이 필요하다.

####  => 각 변수의 특성에 맞는 전처리필요 & 이상값 확인필요


# Raw data Pre-processing
1. percentage_male의 치환
2. "height_m","type2","weight_kg" 결측값의 적절한 치환값

## 1. percentage_male의 치환
* 0은 "여성 몬스터" ~ 100은 "남성 몬스터" 따라서 "중성"인 NaN은 **-1**로 치환하겠다.

In [ ]:
val_length = str(len(data))
pm_change_length = str(len(data[data['percentage_male'].isnull()]))
                 
print("전체 변수 값의 개수 : {0}, 치환해야할 변수 값의 개수 : {1}".format(val_length, pm_change_length))

In [ ]:
## 치환 변수들의 값 치환 : -1 (중성)
data.loc[data['percentage_male'].isnull(),'percentage_male'] = -1

In [ ]:
## 결과확인 : 98개의 변수 치환 완료
print(data[data['percentage_male']==-1].shape)
data[data['percentage_male']==-1].head(3)

## 2. "height_m","type2","weight_kg" 결측값의 적절한 치환값
* 간단한 방법 : mean 또는 누락 값 제거
* 두번째 방법 : 상관성이 높은 변수와의 연관성을 고려한 치환

### 2.1 변수탐색

In [ ]:
## type2 변수의 탐색
type2_nan = len(data[data['type2'].isnull()])
                 
print("전체 변수 값의 개수 : {0}, 치환해야할 변수 값의 개수 : {1}".format(str(len(data)), str(type2_nan)))

type2_dummy_view = pd.concat([data[data['type2'].isnull()].head(3), data[data['type2'].notnull()].head(3)])
type2_dummy_view

### tpye2 변수 탐색결과
* null이 포함된 변수와 null이 포함되지 않은 값들의 비교 에도 특별한 차이점을 찾을 수 없다.
* **name**을 통해 어느정도 유추할 수 있지만 이는 **tpye1**에 언급되기 때문에 큰 의미가 없을것으로 보인다.

### => 따라서, "결측치" 상태로 유지한다.

In [ ]:
## height_m 변수의 탐색
height_m_nan = data[data['height_m'].isnull()]
                 
print("전체 변수 값의 개수 : {0}, 치환해야할 변수 값의 개수 : {1}".format(str(len(data)), str(len(height_m_nan))))

In [ ]:
## weight_kg 변수의 탐색
weight_kg_nan = data[data['weight_kg'].isnull()]
                 
print("전체 변수 값의 개수 : {0}, 치환해야할 변수 값의 개수 : {1}".format(str(len(data)), str(len(weight_kg_nan))))

In [ ]:
## 치환을 위한 상관성이 큰 변수 찾기
data.corr()[['height_m','weight_kg']].T

In [ ]:
## 두 변수 모두 서로 +0.65의 다른 변수들과 비교할때 가장 높은 상관성을 가진다.
print(data.corr()['height_m'].rank(ascending=False)['weight_kg'])
print(data.corr()['weight_kg'].rank(ascending=False)['height_m'])

In [ ]:
## 하지만 두 변수의 null값의 index가 동일하므로 상관변수를 통한 치환은 불가능
print(height_m_nan.index)
print(weight_kg_nan.index)

### height_m , wight_kg 변수 탐색결과
* 두 변수의 NaN 값들이 공통된 index를 가짐을 알 수 있다.
* 따라서 가장 높은 상관성을 통한 치환은 불가능하다.

#### => 차선책으로 2번째로 높은 상관성의 변수를 통해 치환한다. 하지만 두 변수의 상관성이 높으므로 공통으로 높은 변수를 택해야한다

## 2.2 변수치환
* **치환식**

\begin{align}
\ x: base total변수  & & y : weight변수 && or &&  height변수 \\
\end{align}

$$value=\sum_{i=0}^n \frac{(logx-logy)}{n} $$

* value를 구한 후 y변수의 null 값들을 다음식 처럼 치환한다. 

$$ null값 = e^{log(x)   -   value} $$

In [ ]:
## 'base_total' 변수가 hegit_m, weight_kg 변수 모두와 2번째로 상관성이 높으므로 이 변수를 통해 값을 치환한다.
print(data.corr()['height_m'].rank(ascending=False)['base_total'])
print(data.corr()['weight_kg'].rank(ascending=False)['base_total'])

In [ ]:
## base_total은 Metadata에서 정의되지 않았다.
## -> 데이터탐색 결과 무게, 키, 능력치(공격력, 수비력, 속도 등) 과 높은 상관성을 보인다.
## -> 탐색결과 : 'base_total' = attack','defense','hp','sp_attack','sp_defense','speed'의 합으로 보임
## 따라서 base_total을 통해 weight와 heigth의 유추가 신뢰성 있다 판별가능.
pd.DataFrame(data.corr()['base_total']).T

print("base_total 값 : {0} ".format(data.loc[1]['base_total']))
print("pokemoon 능력치 변수들 값의 합 : {0} ".format(data.loc[1][['attack','defense','hp','sp_attack','sp_defense','speed']].sum()))

In [ ]:
## 치환 기준 value 구하기
heigth_notnull = data[data['height_m'].notnull()]
weight_notnull = data[data['weight_kg'].notnull()]

print(heigth_notnull.shape)
mean_value_height = (np.log(heigth_notnull['base_total']) - np.log(heigth_notnull['height_m'])).mean()
mean_value_weight = (np.log(weight_notnull['base_total']) - np.log(weight_notnull['weight_kg'])).mean()
print(mean_value_height)
print(mean_value_weight)

In [ ]:
## Height NaN 값 치환
data.loc[data['height_m'].isnull(),'height_m'] = round(np.exp((abs(np.log(data['base_total']) - mean_value_height))),1)
## weight NaN 값 치환
data.loc[data['weight_kg'].isnull(),'weight_kg'] = round(np.exp((abs(np.log(data['base_total']) - mean_value_weight))),1)

In [ ]:
## NaN였던 18번 index의 치환 값 확인
data.loc[18][['height_m','weight_kg']]

In [ ]:
## NaN 값 치환확인 2
check_height_nan = data[data['height_m'].isnull()]
check_weight_nan = data[data['weight_kg'].isnull()]
                 
print("Height_m 변수의 NaN 값의 개수 : {0}".format(str(len(check_height_nan))))
print("Weight_kg 변수의 NaN 값의 개수 : {0}".format(str(len(check_weight_nan))))

### Result
* type2를 제외한 모든 누락 값들을 치환하였다.

## 3. Outlier 확인
1. 직관적 탐색
2. Box-plot 탐색

### 1. 직관적 탐색을 통한 이상치 확인

In [ ]:
data.describe()

In [ ]:
## capture_rate 변수의 이상치 발견
print(data['capture_rate'][1])
print(data['capture_rate'][773])



In [ ]:
print("바뀌기전 데이터 차원" + str(data.shape))
data = data[data['capture_rate']!='30 (Meteorite)255 (Core)']
print("바뀐 후 데이터 차원" + str(data.shape))

### 2. Box-plot을 통한 이상값 확인
* 그림 whisker(수염) 밖에 있는 부분은 모두 이상값으로 볼것인가?
* outlier 기준 식 = {Q3(75th percentile)-Q1(25th percentile)} * 1.5

![title](http://cfile22.uf.tistory.com/image/2532913754A7F3893809E7)   


In [ ]:
fig, ((ax1,ax2,ax3,ax4),(ax5,ax6,ax7,ax8),(ax9,ax10,ax11,ax12)) = plt.subplots(3,4)
fig.set_size_inches(15,15)

## Boxplot 그리기
sns.boxplot(data=data['attack'], orient="h", palette="Set2",ax=ax1)
sns.boxplot(data=data['base_egg_steps'], orient="h", palette="Set2",ax=ax2)
sns.boxplot(data=data['base_happiness'], orient="h", palette="Set2",ax=ax3)
sns.boxplot(data=data['base_total'], orient="h", palette="Set2",ax=ax4)
sns.boxplot(data=data['defense'], orient="h", palette="Set2",ax=ax5)
sns.boxplot(data=data['experience_growth'], orient="h", palette="Set2",ax=ax6)
sns.boxplot(data=data['height_m'], orient="h", palette="Set2",ax=ax7)
sns.boxplot(data=data['hp'], orient="h", palette="Set2",ax=ax8)
sns.boxplot(data=data['sp_attack'], orient="h", palette="Set2",ax=ax9)
sns.boxplot(data=data['sp_defense'], orient="h", palette="Set2",ax=ax10)
sns.boxplot(data=data['speed'], orient="h", palette="Set2",ax=ax11)
sns.boxplot(data=data['weight_kg'], orient="h", palette="Set2",ax=ax12)

fig.suptitle("Numberic data의 Box-plot 확인", fontsize=16, color='blue')


### Result
* 801 개의 변수밖에 없는데 이상치를 검출하는게 유의미할까?
* 잘못입력된 **오탈자**는 없는 것으로 확인하였다.
* => 또한 각 데이터가 의미하는 것은 포켓몬의 **종류**인데 각기 다른데이터로 이상치를 검출 하는 것은 말이 안된다. 
* => **오탈자**가 없다고 판단되는 상황에서 이 데이터의 이상치는 없는것으로 판단한다.
#### base_total 데이터는 유일하게 완전히 이상값이 없다고 판단되므로 포켓몬의 "능력"을 판단하기 가장 적절한 변수로 생각된다.

In [ ]:
data.to_csv('pokemon_clear_data.csv')

### Pre-processing 결과
* 누락값, 이상값을 처리한 data 저장

# Data Exploration
1. 각 변수들의 의미를 탐색
2. Categorical 변수를(**종족**) 중심으로 데이터 탐색

## 1. 각 변수들의 의미 탐색

* name: The English name of the Pokemon   
* japanese_name: The Original Japanese name of the Pokemon   
* pokedex_number: The entry number of the Pokemon in the National Pokedex   
* percentage_male: The percentage of the species that are male. Blank if the Pokemon is genderless.   
* type1: The Primary Type of the Pokemon   
* type2: The Secondary Type of the Pokemon   
* classification: The Classification of the Pokemon as described by the Sun and Moon Pokedex   
* height_m: Height of the Pokemon in metres
* weight_kg: The Weight of the Pokemon in kilograms
* capture_rate: Capture Rate of the Pokemon
* base_egg_steps: The number of steps required to hatch an egg of the Pokemon
* abilities: A stringified list of abilities that the Pokemon is capable of having
* experience_growth: The Experience Growth of the Pokemon
* base_happiness: Base Happiness of the Pokemon
* against_?: Eighteen features that denote the amount of damage taken against an attack of a particular type
* hp: The Base HP of the Pokemon
* attack: The Base Attack of the Pokemon
* defense: The Base Defense of the Pokemon
* sp_attack: The Base Special Attack of the Pokemon
* sp_defense: The Base Special Defense of the Pokemon
* speed: The Base Speed of the Pokemon
* generation: The numbered generation which the Pokemon was first introduced
* is_legendary: Denotes if the Pokemon is legendary.

    ----------------------------------------------------------------------------------------------------------------
    base_total = attack + defense + hp + sp_attack + sp_defense + speed
    가성비 좋은 종족 : 잡기 쉽고(capture_rate), 키우기 쉽지만(base_egg_steps) 적당한 능력치(base_total)를 가진 종족
    -----------------------------------------------------------------------------------------------------------------
    

## 2. 시각화를 통한 "포캣몬 종족"별 데이터 탐색

In [ ]:
pokemon_data = pd.read_csv('pokemon_clear_data.csv', encoding='ISO-8859-1')
pokemon_data = pokemon_data.drop(['Unnamed: 0'],axis=1)
print(pokemon_data.shape)
pokemon_data.head(3)

In [ ]:
data['type1'].unique()

In [ ]:
def visualization(val,title):
    val.set_title(title, fontsize=18, weight='bold')
    val.tick_params(axis='both', which='major', labelsize=15)

In [ ]:
fig, (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11) = plt.subplots(11,1)
fig.set_size_inches(18,100)

sns.countplot(data=data, x='type1', ax=ax1)
visualization(ax1, '속성별 포캣몬 수')
sns.barplot(data=data, x='type1',y='base_total',ax=ax2)
visualization(ax2,'포캣몬 능력치')
sns.barplot(data=data, x='type1',y='against_fire',ax=ax3)
visualization(ax3,'불 상성')
sns.barplot(data=data, x='type1',y='against_water',ax=ax4)
visualization(ax4,'물 상성')
sns.barplot(data=data, x='type1',y='experience_growth',ax=ax5)
visualization(ax5,'필요경험치')

## 성비 데이터 변경(중성 제거)
data2 = data[data['percentage_male']!=-1]
sns.barplot(data=data2, x='type1',y='percentage_male',ax=ax6)
visualization(ax6,'성비(0~1 : 여성~남성)')
sns.barplot(data=data, x='type1',y='weight_kg',ax=ax7)
visualization(ax7,'몸무게')
sns.barplot(data=data, x='type1',y='height_m',ax=ax8)
visualization(ax8,'키')
sns.barplot(data=data, x='type1',y='is_legendary',ax=ax9)
visualization(ax9,'레전드 포캣몬 유무')
sns.barplot(data=data, x='type1',y='base_egg_steps',ax=ax10)
visualization(ax10,'진화 필요 걸음수')
sns.barplot(data=data, x='type1',y='base_egg_steps',ax=ax11)
visualization(ax11,'포획 성공율')


### Result
* 포캣몬 능력치 : bug 포캣몬이 상대적으로 조금 낮고, dragon, steel 포캣몬이 높다. 나머지 포캣몬의 평균은 비슷한 편이다.   
! flying종족이 편차가 매우 큰걸 볼 수 있다.
* 상성 : 확실히 **"종족"**마다 **반대**되는 상관성을 가지는 상성이 존재한다.   
ex) 불 <-> 물   
* 필요경험치 : fairy종족이 특별히 **낮다** -> 빠른 성장으로 게임을 즐기는 유저 추천
* 성비(**중성제외**) : fairy종족이 확실히 요정 족 답게 "여성캐릭"이 많고, fighting, rock종족이 "남성"이 많다.
* 몸무게, 키 : steel종족이 상대적으로 크다.
* 레전드 포켓몬 유무 : psychic, flying 종족이 많은데 이중, flying종족은 극단적인 **편차**를 가진다. 이는 **포캣몬 능력치**에서 살펴 본 편차 양상과 일치한다. 즉, flying종족은 능력의 **빈부격차**가 매우 크다. -> 대박 or 쪽박
* 진화 필요 결음수 : poison이 제일 낮지만.... 능력치도 낮다... -> 능력치와 성장속도는 확실히 반비례한다. 하지만 **가성비**좋은 종족도 확실히 존재한다.   
ex) dark
* 포획 성공률(잡기 쉬운 정도) : poison이 가장 출현빈도가 높다. psychic이 가성비가 좋은 종족으로 보인다.

```python
!! 가설1 : 사전에 정의된 속성(type1) 즉, "일반적인 상성"과 다른 포캣몬이 있을까??
!! 가설2 : 그렇다면, 예전부터 존재하는 "포캣몬들의 결혼"과 "자식"에 대한 괴담같은 "이종포캣몬!!"이 있지않을까??
```

# 가설 1
## 1. 군집확인

   * 군집화를 통해 **정의된 속성 분류**와 다른 속성에 속하는 포캣몬을 찾아보자.
   * 군집화에서 가장 중요한 것은 분류 **기준**이다.
   * 빈도수, 비율 등 다양한 방법이 있지만, **기존의 속성을 가장 많이 포함하는 군집**을 상성을 통한 포캣몬 분류의 기준으로 삼는다.    
ex) 'fire' 속성의 포켓몬은 군집화 결과 5, 16, 12의 군집으로 각각 45개, 6개, 1개 속하게 된다. 따라서 가장 많은 45개가 속한 **5번**이 새로운 fire 속성의 군집이 된다.
```python
cluster_table[cluster_table['사전에 정의된 속성']=='fire']['상성으로 군집화 한 속성'].value_counts()
# output = 5 : 45, 16 : 6, 12 : 1
```

In [ ]:
from sklearn.cluster import KMeans
import mglearn
from tqdm import tqdm

In [ ]:
element_differ = ['against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water']
cluster_value = data[element_differ]

In [ ]:
## 속성(18개)을 기준으로 군집화
kmeans = KMeans(n_clusters=18, random_state=1)
kmeans.fit(cluster_value)

In [ ]:
## 각각의 포캣몬 별로 "사전에 정의된 속성(Metadata) <-> K-means로 정의한 속성" 을 비교
cluster_table = pd.concat([ pokemon_data[['name','type1']], pd.DataFrame(kmeans.labels_)],axis=1)
cluster_table.columns = ['포캣몬이름','사전에 정의된 속성', '상성으로 군집화 한 속성']
cluster_table

In [ ]:
## 각각의 군집(18개)가 실제 어떤 속성에 해당되는지 알아본다.
## 이때, 기준은 속성의 포캣몬이 "가장 많이 속한" 군집의 번호로한다.

dict_label = {}

for index in tqdm(pokemon_data['type1'].unique()):
    temp = cluster_table[cluster_table['사전에 정의된 속성']==index]['상성으로 군집화 한 속성'].value_counts().index[0]
    
    dict_label[index] = temp
    

In [ ]:
dict_label.items()

### Result
* 18개의 군집으로 속성들을 분류하였지만, 중복되는 속성이 보인다. ex) dark, ice, normal (8번 군집)
* 이는 데이터의 부족과 함께 상성이 일치하는 속성도 있다고 생각하고, 억지로 속성을 나누지 않고 **공통된 군집**으로 포함시킨다.

## 2. 사전적 속성과 군집속성의 비교

### 2.1. 기존과의 차이 확인하기

In [ ]:
## 각각의 사전적 속성의 포켓몬 개체 수
type1_count = pd.DataFrame(pokemon_data['type1'].value_counts()).reset_index()
type1_count.T

In [ ]:
## 새로운 군집에 따른 포켓몬 개체 수
# 10,12,15,16 의 군집은 'other'란 속성으로 정의하여 확실한 사전적 속성이 없는 변수들이다.
cluster_label = pd.DataFrame(kmeans.labels_)
cluster_count = pd.DataFrame(cluster_label[0].value_counts()).reset_index()

for label, c_number in dict_label.items():
    cluster_count.loc[cluster_count['index']==c_number,'index'] = label
    
cluster_count.T

In [ ]:
## Merge()를 사용하여 보기좋게 테이블 합치기
comp_type_table = pd.merge(type1_count, cluster_count, on='index')

## Table 전처리
comp_type_table.loc[comp_type_table['index']=='electric','index'] = 'electric_ground'
comp_type_table.loc[comp_type_table['index']=='psychic','index'] = 'psychic_ghost'
comp_type_table.loc[comp_type_table['index']=='normal','index'] = 'normal_ice_dark'
comp_type_table.columns=['new_type','기존 값','K-means 값']
comp_type_table.index = comp_type_table['new_type']
comp_type_table = comp_type_table.drop(['new_type'],axis=1)

comp_type_table.T

### Result
* 어느정도 군집의 변화가 있는걸 확인할 수 있다.
* 이 차이가나는 포캣몬들을 **이종포캣몬(사전적 분류와 다른 포캣몬)**이라고 정의하자!

### 2.2 이중포켓몬 추출하기
* 먼저, 이중포켓몬인 애들을 추출하자

In [ ]:
dict_label.items()

In [ ]:
cluster_table2 = cluster_table.copy()
cluster_table2.columns = ['poke_name','raw_type','new_type']

## 기존 속성(raw_type)과 군집 속성(new_type)의 비교를 위해 raw_type을 int형인 raw_type2로 치환해준다.
for index, new_type in dict_label.items():
    cluster_table2.loc[cluster_table2['raw_type']==index ,'raw_type2'] = new_type

## 자료형을 int로 맞춰준다.
cluster_table2['raw_type2'] = cluster_table2['raw_type2'].apply(int)

print(cluster_table2.shape)
cluster_table2.head(3)

In [ ]:
## 이종 or 동일로 "이종포켓몬"을 정의한다.
cluster_table2['check'] = cluster_table2['new_type']-cluster_table2['raw_type2']
cluster_table2.loc[cluster_table2['check']==0,'check'] = '순종'
cluster_table2.loc[cluster_table2['check']!='순종','check'] = '이종'

print(cluster_table2.shape)
cluster_table2.head(10)

In [ ]:
fusion_pokemon = cluster_table2[cluster_table2['check'] == '이종']
print(fusion_pokemon.shape)
pure_pokemon = cluster_table2[cluster_table2['check'] == '순종']
print(pure_pokemon.shape)
pure_pokemon.head(3)

### Result
* '이종'과 '순종' 포켓몬을 256, 544개로 구분하였다.
* 가설의 검증이 필요

# 3. 가설에 따른 데이터 분석
1. 가설검증   

2. "이종"포켓몬의 특징을 알아보고, 일반적인 포켓몬과의 차이를 알아보자.

## 3.0 (1) 이종 포켓몬 데이터 전처리

In [ ]:
## merge를 위해 특정 칼럼 명 변경
fusion_pokemon=fusion_pokemon.rename(columns = {'poke_name':'name'})

In [ ]:
## 기존 능력치 데이터와 추출한 '이종'포켓몬 데이터의 병합
## 병합기준 : 포켓몬이름
fusion_poke_hap = pd.merge(fusion_pokemon,pokemon_data, on ='name')

print(fusion_poke_hap.shape)
fusion_poke_hap.head(3)

In [ ]:
dict_label2 = {'bug': 2,'dark_ice_normal':8,'dragon': 4,'electric_ground': 11,'fairy': 7,'fighting': 0,'fire': 5,'flying': 13,
               'ghost_psychic': 3,'grass': 6,'poison': 17,'rock': 14,'steel': 9,'water': 1,'other1':10, 'other2':12, 'other3':15,'other4':16}

In [ ]:
for index, number in dict_label2.items():
    fusion_poke_hap.loc[fusion_poke_hap['new_type'] == number,'new_type'] = index

print(fusion_poke_hap.shape)
fusion_poke_hap.head(5)

In [ ]:
## 전처리
for index in ['other1','other2','other3','other4']:
    fusion_poke_hap.loc[fusion_poke_hap['new_type'] == index, 'new_type'] = -1
    
fusion_poke_hap = fusion_poke_hap[fusion_poke_hap['new_type'] != -1]

print(fusion_poke_hap.shape)
fusion_poke_hap.head(5)

## 3.0 (2) 순수포켓몬 데이터 전처리

In [ ]:
## merge를 위해 특정 칼럼 명 변경
pure_pokemon=pure_pokemon.rename(columns = {'poke_name':'name'})

In [ ]:
## 기존 능력치 데이터와 추출한 '동종'포켓몬 데이터의 병합
## 병합기준 : 포켓몬이름
pure_poke_hap = pd.merge(pure_pokemon,pokemon_data, on ='name')

print(pure_poke_hap.shape)
pure_poke_hap.head(3)

In [ ]:
dict_label2 = {'bug': 2,'dark_ice_normal':8,'dragon': 4,'electric_ground': 11,'fairy': 7,'fighting': 0,'fire': 5,'flying': 13,
               'ghost_psychic': 3,'grass': 6,'poison': 17,'rock': 14,'steel': 9,'water': 1}

In [ ]:
## raw_type2는 사전적 속성을 숫자로 변경한것이다. 

for index, number in dict_label2.items():
    pure_poke_hap.loc[pure_poke_hap['raw_type2'] == number,'raw_type2'] = index

print(pure_poke_hap.shape)
pure_poke_hap.head(5)

## 3.1 가설검증
- 가설 : 사전적으로 정의된 포켓몬과 다른 "이종" 포켓몬이 존재할 것이다.
- 전제1 : 포켓몬은 결혼을 통해 자식을 낳는 **설정이 있다.**
- 전제2 : 따라서 다른 종간의 번식을 통해 생긴 **"이종"**포켓몬은 사전적으로 정의된 속성과 **다른 능력치**를 가지고 있을것이다.(종족의 섞였기 때문에)
* 정의 : 능력치(base_total) = attack + defense + hp + sp_attack + sp_defense + speed
    #### =>  귀무가설 : 이종, 순종 포켓몬의 능력치는 같을 것이다.

In [ ]:
## 능력치 비교 시각화
def visualization(val,title):
    val.set_title(title, fontsize=18, weight='bold')
    val.tick_params(axis='both', which='major', labelsize=15)
    val.set_xlim(0,600)
    val.set_ylabel('')

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(18,10)


sns.barplot(data=pure_poke_hap.sort_values('raw_type2'), y='raw_type2',x='base_total',ax=ax1)
visualization(ax1,'순종 포켓몬 능력치')

sns.barplot(data=fusion_poke_hap.sort_values('new_type'), y='new_type',x='base_total',ax=ax2)
visualization(ax2,'이종 포켓몬 능력치')

In [ ]:
## 순종, 이종 포켓몬 능력치 상세 비교
mean_pure = pd.DataFrame(pure_poke_hap.groupby('raw_type2')['base_total'].mean().apply(int))
mean_fusion = pd.DataFrame(fusion_poke_hap.groupby('new_type')['base_total'].mean().apply(int))
performance_comp = pd.concat([mean_pure, mean_fusion],axis=1)
performance_comp.columns = ['순종','이종']
performance_comp.T

#### 통계적 검증(일원배치법=ANOVA)

In [ ]:
## 능력치의 평균을 이용하므로 t-test를 통한 검정

import scipy.stats as stats
stats.ttest_ind(performance_comp['이종'], performance_comp['순종'])   

### Result
* 가설검정결과(90%의 신뢰구간 : 데이터의 양이 매우 적으므로)
    - p-value가 0.06으로 0.1보다 작아서 **귀무가설을 기각**할 수 있다.
* 능력치를 비교한 결과
    1. 순종의 능력치 우위 : dark_ice_normal, fairy, grass, rock
    2. 이종의 능력치 우위 : bug, dragon, eletric_ground, fighting, fire, flying, poison, steel
    3. 비슷한 수준 : ghost_psychic, water
    
#### => 가설검증을 통해 '이종'포켓몬의 존재를 확인하였다.

## 3.2 이종포켓몬과 순종포켓의 차이
* 데이터탐색을 통한 확인

In [ ]:
pokemon_data.columns

In [ ]:
def visualization(val,title):
    val.set_title(title, fontsize=18, weight='bold')
    val.tick_params(axis='both', which='major', labelsize=15)
    if(val == ax1 or val==ax2):
        val.set_xlim(0,600)
    val.set_ylabel('')
    if(val == ax3 or val==ax4):
        val.set_xlim()

In [ ]:
## 능력치 비교 시각화


fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2)
fig.set_size_inches(18,30)

## 능력치 비교
sns.barplot(data=pure_poke_hap.sort_values('raw_type2'), y='raw_type2',x='base_total',ax=ax1)
visualization(ax1,'순종 포켓몬 능력치')
sns.barplot(data=fusion_poke_hap.sort_values('new_type'), y='new_type',x='base_total',ax=ax2)
visualization(ax2,'이종 포켓몬 능력치')

##가성비 비교(가성비 = 능력치/진화필요 횟수)
test = pure_poke_hap['base_total']/pure_poke_hap['base_egg_steps']
sns.barplot(data=pure_poke_hap.sort_values('raw_type2'), y='raw_type2',x=test,ax=ax3)
visualization(ax3,'순종 포켓몬 가성비')
sns.barplot(data=fusion_poke_hap.sort_values('new_type'), y='new_type',x=test,ax=ax4)
visualization(ax4,'이종 포켓몬 가성비')

##성장성 비교(경험치 증가율) 높을 수록 미래의 성장성이 높다.(=키우기 힘들다.)
sns.barplot(data=pure_poke_hap.sort_values('raw_type2'), y='raw_type2',x='experience_growth',ax=ax5)
visualization(ax5,'순종 포켓몬 성장성')
sns.barplot(data=fusion_poke_hap.sort_values('new_type'), y='new_type',x='experience_growth',ax=ax6)
visualization(ax6,'이종 포켓몬 성장성')

### Result
* 탐색적 분석을 통해 **두 가지 종류의 포켓몬을 추천해 줄 수 있다.**
    1. 빠른성장으로 재미를 보고 싶은 유저에게 추천하는 속성
    2. 미래의 성장성을 기대하는 유저에게 추천하는 속성
    3. 기타 특이 포켓몬..

## 3.2 (1) 빠른성장 & 가성비좋은 포켓몬 <이종포켓몬>
* 순종포켓몬은 **"전형적인 상성"**을 가지고 있으므로 ex) 물은 불에 약하다. 싸움의 승리를 위해서는 **이종** 포켓몬으로 변수를 주는것이 좋다.
* 따라서 "이종포켓몬"을 기준으로 좋은 포켓몬을 찾아보자

In [ ]:
## 순종, 이종 포켓몬 능력치 상세 비교
mean_pure = pd.DataFrame(pure_poke_hap.groupby('raw_type2')['base_total'].mean().apply(int))
mean_fusion = pd.DataFrame(fusion_poke_hap.groupby('new_type')['base_total'].mean().apply(int))
performance_comp = pd.concat([mean_pure, mean_fusion],axis=1)
performance_comp.columns = ['순종능력치','이종능력치']

performance_comp['compare'] = performance_comp['순종능력치']-performance_comp['이종능력치']
performance_comp['변동수준'] = abs(performance_comp['compare'])

for temp in performance_comp.index:
    if(performance_comp[performance_comp.index==temp]['compare'].values[0] > 0):
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '순종승'
    else:
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '이종승'
        
performance_comp.T

In [ ]:
## 순종, 이종 포켓몬 가성비 상세 비교
pure_poke_hap['pure_gaseong'] = pure_poke_hap['base_total']/pure_poke_hap['base_egg_steps']
fusion_poke_hap['fusion_gaseong'] = fusion_poke_hap['base_total']/fusion_poke_hap['base_egg_steps']

mean_pure = pd.DataFrame(pure_poke_hap.groupby('raw_type2')['pure_gaseong'].mean()*100)
mean_fusion = pd.DataFrame(fusion_poke_hap.groupby('new_type')['fusion_gaseong'].mean()*100)
performance_comp = pd.concat([mean_pure, mean_fusion],axis=1)
performance_comp.columns = ['순종가성비','이종가성비']

performance_comp['compare'] = performance_comp['순종가성비']-performance_comp['이종가성비']
performance_comp['변동수준'] = abs(performance_comp['compare'])

for temp in performance_comp.index:
    if(performance_comp[performance_comp.index==temp]['compare'].values[0] > 0):
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '순종승'
    else:
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '이종승'


performance_comp.T

### Result
* 능력치 와 가성비 모두 "이종" 이 높은 종족 ( bug, dragon, eletric_ground, fighting, ghost_psychic, steel)
* 그중, 가성비가 가장 좋은 종족 : electric_gound / 능력치가 가장 좋은 종족 : fighting

#### => 완전 빠른 성장이 목적 =  electric_ground 속성의 포켓몬 추천 (능력치 순위 : 6위)
#### => 적당한 성장 but 강한 포켓몬의 목적 = fighting 속성의 포켓몬 추천 (가성비 순위 : 8위) 

## 3.2 (2) 미래의 성장성이 높은 포켓몬

In [ ]:
## 순종, 이종 포켓몬 능력치 상세 비교
## 높을수록 성장 경험치가 많이 필요하다 = 키우기 힘들다. but 후반에 강해질 확률이 높다.

mean_pure = pd.DataFrame(pure_poke_hap.groupby('raw_type2')['experience_growth'].mean().apply(int))
mean_fusion = pd.DataFrame(fusion_poke_hap.groupby('new_type')['experience_growth'].mean().apply(int))
performance_comp = pd.concat([mean_pure, mean_fusion],axis=1)
performance_comp.columns = ['순종성장성','이종성장성']

performance_comp['compare'] = performance_comp['순종성장성']-performance_comp['이종성장성']
performance_comp['변동수준'] = abs(performance_comp['compare'])

for temp in performance_comp.index:
    if(performance_comp[performance_comp.index==temp]['compare'].values[0] > 0):
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '순종승'
    else:
        performance_comp.loc[performance_comp.index==temp, 'compare'] = '이종승'
        
performance_comp.T

#### Result
* 후반만 바라보는 유저는 느리더라도 무조건 강할걸 좋아한다.
* 성장성 1위 : fighting (능력치 순위 : 1위)


#### => fighting종족 강력 추천

# 최종 포켓몬
* fighting 종족 중 최강의 포켓몬

In [ ]:
final_pokemon = fusion_poke_hap[fusion_poke_hap['new_type'] == 'fighting']
print(final_pokemon.shape)
final_pokemon

In [ ]:
final_pokemon['gaseong'] = final_pokemon['base_total']/final_pokemon['base_egg_steps']

In [ ]:
final_pokemon[['name','base_total','experience_growth','gaseong','capture_rate']]

## 4. 최종결과 & 아쉬운점

* **추천포켓몬**
    - "Gallade" : 얼레이드, 실제 포켓몬 세계관에서도 강력한 친구!
    
    
* **최종결과** 
    - 탐색적 방법을 사용하여 "이종"포켓몬의 존재를 확인하였다.


* **아쉬운점**
    - 데이터 양의 부족으로 **효과적인 군집**을 이루지 못했다.
    - 군집의 결과가 완벽하다 할 수 없어 분석의 빈틈이 많다.....
    - 데이터의 양이 너무 부족하여 통계적 검증부분에서 근거가 부족하였다. 또한, **머신러닝**을 활용하여 인사이트를 도출하는데 한계가 있었다. 만약, 801종의 포켓몬별로 각각 1개의 샘플이 아닌 여러 데이터가 있었다면 더 좋은 분석이 가능했을거 같아 아쉬움이 남는다...